In [1]:
function repeatlength(A, i)
    k = 1
    @inbounds while i + k ≤ lastindex(A) && isequal(A[i+k], A[i])
        k += 1
    end
    k
end

"""
    RepVal(A)
Iterator generating tuples of repeat length of `v` and value `v` in `A`
"""
struct RepVal{T} A::T end
Base.IteratorSize(::Type{RepVal{T}}) where T = Base.SizeUnknown()
Base.eltype(x::RepVal) = Tuple{Int, eltype(x.A)}

Base.iterate(x::RepVal) = iterate(x, firstindex(x.A))
function Base.iterate(x::RepVal, i::Int)
    i > lastindex(x.A) && return nothing
    k = repeatlength(x.A, i)
    (k, x.A[i]), i + k
end

maxrep_maxval(A) = maximum(RepVal(A))
negrep((k, v)) = (-k, v)
maxrep_minval(A) = negrep(minimum(negrep, RepVal(A)))

maxrep_minval (generic function with 1 method)

In [2]:
A = [1, 2, 3, 3, 3, 1, 1, 1, NaN, NaN, NaN, 2, 2, 3]
@show A
RepVal(A) |> collect

A = [1.0, 2.0, 3.0, 3.0, 3.0, 1.0, 1.0, 1.0, NaN, NaN, NaN, 2.0, 2.0, 3.0]


7-element Vector{Tuple{Int64, Float64}}:
 (1, 1.0)
 (1, 2.0)
 (3, 3.0)
 (3, 1.0)
 (3, NaN)
 (2, 2.0)
 (1, 3.0)

In [3]:
maxrep_maxval(A)

(3, NaN)

In [4]:
maxrep_minval(A)

(3, 1.0)

In [5]:
@code_warntype iterate(RepVal(A), 1)

Variables
  #self#::Core.Const(iterate)
  x::RepVal{Vector{Float64}}
  i::Int64
  k::Int64

Body::Union{Nothing, Tuple{Tuple{Int64, Float64}, Int64}}
1 ─       Core.NewvarNode(:(k))
│   %2  = Base.getproperty(x, :A)::Vector{Float64}
│   %3  = Main.lastindex(%2)::Int64
│   %4  = (i > %3)::Bool
└──       goto #3 if not %4
2 ─       return Main.nothing
3 ─ %7  = Base.getproperty(x, :A)::Vector{Float64}
│         (k = Main.repeatlength(%7, i))
│   %9  = k::Int64
│   %10 = Base.getproperty(x, :A)::Vector{Float64}
│   %11 = Base.getindex(%10, i)::Float64
│   %12 = Core.tuple(%9, %11)::Tuple{Int64, Float64}
│   %13 = (i + k)::Int64
│   %14 = Core.tuple(%12, %13)::Tuple{Tuple{Int64, Float64}, Int64}
└──       return %14
